# Simple RAG (Retrieval-Augmented Generation) System for CSV Files

## Overview

This code implements a basic Retrieval-Augmented Generation (RAG) system for processing and querying CSV documents. The system encodes the document content into a vector store, which can then be queried to retrieve relevant information.

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

## Key Components

1. Loading and spliting csv files.
2. Vector store creation using [FAISS](https://engineering.fb.com/2017/03/29/data-infrastructure/faiss-a-library-for-efficient-similarity-search/) and OpenAI embeddings
3. Retriever setup for querying the processed documents
4. Creating a question and answer over the csv data.

## Method Details

### Document Preprocessing

1. The csv is loaded using langchain Csvloader
2. The data is split into chunks.


### Vector Store Creation

1. OpenAI embeddings are used to create vector representations of the text chunks.
2. A FAISS vector store is created from these embeddings for efficient similarity search.

### Retriever Setup

1. A retriever is configured to fetch the most relevant chunks for a given query.

## Benefits of this Approach

1. Scalability: Can handle large documents by processing them in chunks.
2. Flexibility: Easy to adjust parameters like chunk size and number of retrieved results.
3. Efficiency: Utilizes FAISS for fast similarity search in high-dimensional spaces.
4. Integration with Advanced NLP: Uses OpenAI embeddings for state-of-the-art text representation.

## Conclusion

This simple RAG system provides a solid foundation for building more complex information retrieval and question-answering systems. By encoding document content into a searchable vector store, it enables efficient retrieval of relevant information in response to queries. This approach is particularly useful for applications requiring quick access to specific information within a csv file.

import libries

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [1]:
# Install required packages
# !pip install faiss-cpu langchain langchain-community langchain-openai pandas python-dotenv

In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from pathlib import Path
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# CSV File Structure and Use Case
The CSV file contains dummy customer data, comprising various attributes like first name, last name, company, etc. This dataset will be utilized for a RAG use case, facilitating the creation of a customer information Q&A system.

In [3]:
import os
import sys
from dotenv import load_dotenv
# Load environment variables from a .env file
load_dotenv()

sys.path.append(os.path.abspath('../../RAG_TECHNIQUES/'))

print(os.getcwd())
sys.path[-1]


# Download required data files
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
# !wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
# !wget -O data/customers-100.csv https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/customers-100.csv


/Users/abhinandanpathak/Documents/Learning/Learning/Gen AI/RAG/RAG_Techniques/all_rag_techniques


In [4]:
import pandas as pd

file_path = ('../data/customers-100.csv') # insert the path of the csv file
data = pd.read_csv(file_path)

#preview the csv file
data.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


load and process csv data

In [5]:
data.shape

(100, 12)

In [6]:
loader = CSVLoader(file_path=file_path)
docs = loader.load_and_split()

In [7]:
print(docs[0].to_json()["kwargs"]["page_content"])

Index: 1
Customer Id: DD37Cf93aecA6Dc
First Name: Sheryl
Last Name: Baxter
Company: Rasmussen Group
City: East Leonard
Country: Chile
Phone 1: 229.077.5154
Phone 2: 397.884.0519x718
Email: zunigavanessa@smith.info
Subscription Date: 2020-08-24
Website: http://www.stephenson.com/


Initiate faiss vector store and openai embedding

In [10]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from helper_functions import get_langchain_embedding_provider, EmbeddingProvider

embeddings = get_langchain_embedding_provider(EmbeddingProvider.AZURE_OPENAI) # Instantiate my Embedding 
index = faiss.IndexFlatL2(3072) # Creates a empty Index of Dimension 3072
vector_store = FAISS( # Instantiate In-Memory Vector Store
    embedding_function=embeddings, # To Embed User Query and Docs
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

Add the splitted csv data to the vector store

In [11]:
vector_store.add_documents(documents=docs)

['06a10535-bae2-4956-89d4-90b9d13c0c4c',
 'd79078a0-9f92-460b-a371-e876eafb93db',
 'abbd74fd-33eb-4249-9c10-1748ffa7644a',
 'b404ffa7-1cb8-47f1-9271-9518eb5079dc',
 '4655dfca-cc95-44be-876b-c35f531ac7cd',
 '22ce65ae-36b6-4cfd-b6a5-aa5b1ab7535b',
 'e85a4197-3ace-40a1-93ef-c0d4c696a94e',
 'acc5f56f-5e0f-4c0d-97bd-ec7a6c2e6825',
 'cf023f62-97f4-4cc0-aeb4-01c884e45ddc',
 '7622bad3-9b89-41dc-9dc6-5f0fd86e61b5',
 '6c37b8d6-5af7-41cf-85d9-bd25ab323b87',
 '5fb1d702-8310-4cd0-ab68-c5abdac5e93f',
 'b3326aeb-2e63-4319-9479-55976ef1aeee',
 '0af09b63-98e6-428a-8de4-7dd1d30b1269',
 'efa2cec7-9c44-48a9-bb41-ac6795db4ead',
 '5ff17a0c-5d74-4f42-98bd-d7ab72f200c9',
 'ec598135-0086-4b62-97d8-4469cbbbeea7',
 '5bf0b967-c1d0-4578-84fc-5e9ebe629ff3',
 '678866b4-453c-40a3-9a54-bc590b968c68',
 '59184e12-c686-4362-b579-13a681022217',
 '0dcadbf3-f06d-4328-975e-17c564ab0367',
 'da183ea1-2611-44ec-8ea4-b6594bc093b1',
 'ce36ed85-2482-410c-b56b-01e5fd55b7a3',
 'e1003440-6535-4197-99d4-6ea4bce9b783',
 '21134f10-86ff-

Create the retrieval chain

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

retriever = vector_store.as_retriever()

# Set up system prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    
])

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Query the rag bot with a question based on the CSV data

In [15]:
answer= rag_chain.invoke({"input": "which company does sheryl Baxter work for?"})
answer['answer']

'Sheryl Baxter works for Rasmussen Group.'

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--simple-csv-rag)